In [1]:
import warnings
warnings.filterwarnings('ignore')

In [105]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [135]:
df = pd.read_csv('botswana.tsv', sep='\t')
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [136]:
df['religion'].value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [137]:
df.shape, df.dropna().shape

((4361, 15), (1834, 15))

In [138]:
df['nevermarr'] = df['agefm'].isna()
df.drop('evermarr', axis='columns', inplace=True)
df['agefm'] = df['agefm'].fillna(0)
print(df['heduc'].isna().sum())
df[df['nevermarr'] == 1]['heduc'] = df[df['nevermarr'] == 1]['heduc'].fillna(-1)
print(4361 - df['heduc'].isna().sum())

2405
1956


In [139]:
na_features = ['idlnchld', 'heduc', 'usemeth']
for feature in na_features:
    df[feature + '_noans'] = df[feature].isna()
df['idlnchld'].fillna(-1, inplace=True)
df['heduc'].fillna(-2, inplace=True)
df['usemeth'].fillna(-1, inplace=True)
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-2.0,1,1.0,1.0,1.0,1.0,True,False,True,False
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,False,False,False,False
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,False,False,False,False
3,0,24,12,other,2.0,1.0,0.0,0.0,-2.0,1,1.0,1.0,1.0,1.0,True,False,True,False
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,False,False,False,False


In [140]:
df.dropna(subset=['knowmeth', 'electric', 'radio', 'tv', 'bicycle'], inplace=True)

In [141]:
df.shape[0] * df.shape[1]

78264

In [142]:
y = df['ceb']
X = df.drop('ceb', axis='columns')
y.shape, X.shape

((4348,), (4348, 17))

In [143]:
m1 = smf.ols('ceb ~ ' + ' + '.join(X.columns), data=df)
fitted1 = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 01 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:10:31   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [144]:
sms.het_breuschpagan(fitted.resid, fitted.model.exog)

(1112.4700385717551,
 1.1197458896528959e-228,
 106.41517187062958,
 3.59261131253213e-265)

In [145]:
X.columns

Index(['age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth', 'agefm',
       'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle', 'nevermarr',
       'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [146]:
cols = ['age', 'educ', 'idlnchld', 'knowmeth', 'usemeth', 'agefm',
        'heduc', 'urban', 'electric', 'bicycle', 'nevermarr',
        'idlnchld_noans', 'heduc_noans', 'usemeth_noans']
m2 = smf.ols('ceb ~ ' + ' + '.join(cols), data=df)
fitted2 = m2.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 01 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:10:39   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [150]:
fitted1.compare_f_test(fitted2)

/usr/local/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:2018: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(0.9192357784633435, 0.46723055472730823, 5.0)

In [133]:
sms.het_breuschpagan(fitted.resid, fitted.model.exog)

(1112.4700385717551,
 1.1197458896528959e-228,
 106.41517187062958,
 3.59261131253213e-265)

In [66]:
df.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [151]:
cols = ['age', 'educ', 'idlnchld', 'knowmeth',
        'agefm', 'heduc', 'urban', 'electric', 'bicycle',
        'nevermarr', 'idlnchld_noans', 'heduc_noans']
m3 = smf.ols('ceb ~ ' + ' + '.join(cols), data=df)
fitted3 = m3.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 01 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:14:40   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [152]:
fitted2.compare_f_test(fitted3)

/usr/local/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:2018: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(92.89058230109757, 3.1552009480386492e-40, 2.0)